In [2]:
## The neuropath data at Emory ADRC is currently not particularly well organized
## and naming schema is inconsistent.  Working on developing a schema 
import girder_client
import os,sys, json
import dagSecrets as dgs
from collections import Counter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# import pandas as pd
import pickle
import brain_region_maps as brm

%load_ext autoreload
%autoreload 2
import pandas as pd
import adrcSchemaHelpers as hlprs
import brain_region_maps as brn
gc = girder_client.GirderClient(apiUrl=dgs.cbApiUrl)
gc.authenticate(apiKey=dgs.cbApiKey)


from pivottablejs import pivot_ui

In [3]:
npPickleFile = 'npDemoData.pickle'
savePickle = True
reload = False

if os.path.isfile(npPickleFile) and not reload:
    with open(npPickleFile,"rb") as fp:
        npSlideDict=  pickle.load(fp)
else:
    print("RELOADING DATA")
    npSlideDict = hlprs.getADRCcollection(gc,rootCollectionName='ARMIN_Working_Cases')
    if savePickle:
        with open(npPickleFile,"wb") as fp:
            pickle.dump(npSlideDict,fp)

RELOADING DATA


In [26]:
dfd = []
needsMap = []

for k in npSlideDict:
    yr = int(k.split("_")[0])
    caseId = k.split("_")[1]
    caseSet = npSlideDict[k]
    
    if caseId not in brn.MAP and yr>2014:
        dfd.append({'Year':yr,'caseID':caseId,'totalSlidesInDir':len(caseSet),'validBrainMap':'No'})
        needsMap.append(caseId)
    elif caseId in brn.MAP:
        dfd.append({'Year':yr,'caseID':caseId,'totalSlidesInDir':len(caseSet),'validBrainMap':'Yes'})





In [30]:
#pivot_ui(pd.DataFrame(dfd))
print(len(needsMap))
needsMap

28


['E16-128',
 'E16-15',
 'E16-24',
 'E16-27',
 'E16-33',
 'E16-34',
 'OS160608',
 'OS161018',
 'OS',
 'OS',
 'E18-115',
 'E18-70',
 'E18-86',
 'OS-180213',
 'E19-18',
 'E19-27',
 'E19-31',
 'E19-70',
 'E19-9',
 'E19-97',
 'E20-55',
 'E20-56',
 'E20-63',
 'E20-71',
 'E20-72',
 'E20-73',
 'E20-9',
 'E21-3']

In [15]:
dfd

[{'Year': 2014, 'caseID': 'E14-06', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-09', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-104', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-106', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-111', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-115', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-12', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-122', 'totalSlidesInDir': 48},
 {'Year': 2014, 'caseID': 'E14-123', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-133', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-134', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-14', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-161', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-22', 'totalSlidesInDir': 125},
 {'Year': 2014, 'caseID': 'E14-24', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E14-26', 'totalSlidesInDir': 0},
 {'Year': 2014, 'caseID': 'E

In [26]:
## meta.region  meta.stain  meta.subject
basicSlideInfo = []
mdKeys = ['region','stain','subject','brain_region']

for subjyr in npSlideDict:
    for s in npSlideDict[subjyr]:
        slideData = {'folderId': subjyr,'slideName':s['name'],'itemId':s['_id']}
        
        for k in mdKeys:
            if k in s['meta']:
                slideData[k] = s['meta'][k]
        basicSlideInfo.append(slideData)
pd.DataFrame(basicSlideInfo).to_csv("adrcInitialMetadataExport.csv")

In [27]:
npByYearCase = {}
for k in npSlideDict:
    try:
        (year,caseId) = k.split("_")
        if year not in npByYearCase:
            npByYearCase[year]= []
            
        npByYearCase[year].append(caseId)
        
    except:
        print("Weird pattern for",k)

In [58]:
from ipywidgets import Dropdown, interactive, HBox, interactive_output
from collections import Counter
npYearList = list(npByYearCase.keys())

dpYear = Dropdown(options=npYearList,description='Year')
dpCaseId = Dropdown(options=npByYearCase[npYearList[0]],description='CaseId')


def on_change(*args):
    value = dpYear.value
    dpCaseId.options = npByYearCase[value]
dpYear.observe(on_change, names='value')

def analyzeNpCase(year, caseId):
    print("Gutman needs a bath")
    ### This will perform an analysis of a folder of images to see the number of slides 
    ## And check the metadata
    npKey = "%s_%s" % ( year,caseId)
    slideList = npSlideDict[npKey]
    
    ## Keys of interest
    koi = ['region','regionId','subject','stain','Stain','Region']
    slideKeys = {}
    for k in koi:
        slideKeys[k]= []
    
    caseMeta = [] ## 
    d=caseMeta ## Exposing this as a global
    
    for s in slideList:
        meta = s['meta']
        for k in meta:
            if k in koi:
                slideKeys[k].append(meta[k])
        caseMeta.append( hlprs.validateSlide(s,caseId) )
                
                
    ## Now go through and make things into counters so it's easier to see
    for k in koi:
        slideKeys[k] = Counter(slideKeys[k])
                
    print("Unique RegionId Count: %d \t Distinct SubjID: \t%d"% (len(slideKeys['region']),len(slideKeys['subject'])))
    print("Total slides found: %d" % len(slideList))  ## Will add in schema maps as well
    df = pd.DataFrame(caseMeta)
#     print(df)
    if 'region' in df.columns:
        print(df.region.value_counts())
    else:
        print("region tag not found..")
        print([s['name'] for s in slideList])

    
ui = HBox([dpYear,dpCaseId])
w = interactive_output(analyzeNpCase, {'year':dpYear, 'caseId':dpCaseId})
display(ui,w)

## Key I am interested in is npSchema going forward
## Need to figure out what to do with control slides..  will add a key to the Schema that's optional called controlSlide


Output()

In [ ]:
brm.MAP['A18-07']

In [49]:
## LEt's examine a specific case...  2020_E20-106
import re

caseId = '2020_E20-106'
npSlideDict[caseId]

adrcName_re = re.compile('(?P<subject>.\d\d-\d+)_(?P<blockID>\d+) (?P<stain>.*)\.[svs|ndpi]')

## Let me look at the filenames in this dictionary
for slideName in [x['name'] for x in npSlideDict[caseId]]:
    m =  adrcName_re.search(slideName)
    if m:
        print(slideName,m.groupdict())
    elif ('CON' in slideName.upper() ):  ### This is probably a control slide
        print("Control slide?",slideName)
    else:
        print("Shit nothing works",slideName)

Control slide? Biels con.svs_thumb_1024.jpg
E20-106_1 AB.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '1', 'stain': 'AB'}
E20-106_1 Biels.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '1', 'stain': 'Biels'}
E20-106_1 HE.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '1', 'stain': 'HE'}
E20-106_1 Syn.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '1', 'stain': 'Syn'}
E20-106_1 Tau.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '1', 'stain': 'Tau'}
E20-106_1 pTDP.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '1', 'stain': 'pTDP'}
E20-106_10 AB.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '10', 'stain': 'AB'}
E20-106_10 HE.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '10', 'stain': 'HE'}
E20-106_10 Syn.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '10', 'stain': 'Syn'}
E20-106_10 Tau.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '10', 'stain': 'Tau'}
E20-106_11 HE.svs_thumb_1024.jpg {'subject': 'E20-106', 'blockID': '11

In [57]:
brm.MAP['A20-11']

{'1': 'Hippocampus',
 '2': 'Amygdala',
 '3': 'Frontal cortex',
 '4': 'Temporal cortex',
 '5': 'Parietal cortex',
 '6': 'Occipital cortex',
 '7': 'Left perirolandic cortex',
 '8': 'Cingulate cortex',
 '9': 'Insular cortex',
 '10': 'Anterior basal ganglia',
 '11': 'Posterior basal ganglia',
 '12': 'Hypothalamus',
 '13': 'Thalamus, with subthalamic nucleus',
 '14': 'Midbrain',
 '15': 'Pons',
 '16': 'Medulla',
 '17': 'Cerebellum and dentate nucleus',
 '18': 'Upper spinal cord',
 '19': 'Mid-level spinal cord',
 '20': 'Lower spinal cord'}